In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython import display

In [2]:
# For reporducibility
torch.manual_seed(1)

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [5]:
print('e^1 equals:', torch.exp(torch.FloatTensor([[1]])))

e^1 equals: tensor([[2.7183]])


In [6]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [7]:
print(hypothesis.squeeze().detach())
print(hypothesis.shape)

tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000])
torch.Size([6, 1])


### Sigmoid
 - torch.sigmoid() ==> function
 - Caculate a probability from logit
 - 1 / (1 + exp(-x))

In [8]:
-(y_train[0] * torch.log(hypothesis[0]) + (1 - y_train[0]) * torch.log(1 - hypothesis[0])).detach()

tensor([0.6931])

In [9]:
optimizer = optim.SGD([W, b], lr=1.)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    display.clear_output(wait=True)
    if epoch % 100 == 0 and epoch is not 0:
        print("Epoch: {:4d}/{} Cost: {:.6f}".format(
            epoch, nb_epochs, cost.item()
        ))

Epoch: 1000/1000 Cost: 0.019852


In [10]:
from torch.utils.data import DataLoader, Dataset

class Data(Dataset):
    def __init__(self):
        self.x = x_data
        self.y = y_data
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x[idx])
        y = torch.FloatTensor(self.y[idx])
        
        return x, y
    
dataset = Data()

In [11]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
model = Model()

In [12]:
dataload = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True
)

In [13]:
optimizer = optim.SGD(model.parameters(), lr=1.)
nb_epochs = 100

for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataload):
        x_train, y_train = samples
        
        hypothesis = model(x_train)
        cost = F.binary_cross_entropy(hypothesis, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        display.clear_output(wait=True)
        if epoch % 20 == 0 and epoch is not 0:
            prediction = hypothesis >= torch.FloatTensor([0.5])
            correct_prediction = prediction.float() == y_train
            accuracy = correct_prediction.sum().item() / len(correct_prediction)
            
            print("Epoch: {:4d}/{} Batch: {}/{} Cost: {:.6f}, Acc: {:2.2f}%".format(
                epoch, nb_epochs, batch_idx+1, len(dataload), cost.item(), accuracy*100
            ))

Epoch:  100/100 Batch: 3/3 Cost: 0.049418, Acc: 100.00%
